In [35]:
from dotenv import load_dotenv
import re
import json
import os
import redis
from tmdbv3api import TMDb, TV, Season, Episode
import xml.etree.ElementTree as ET

load_dotenv()

True

In [28]:
r = redis.Redis(host='localhost', port=6379, decode_responses=True)

Test Key:

In [7]:
# Example: Set and get a key
r.set('key', 'value')

True

In [4]:
print(r.get('key'))  # b'value'

value


In [10]:
# Add the media file metadata from the NFO file
metadata = {
    'plot': 'In 2071, roughly fifty years after an accident with a hyperspace gateway made the Earth almost uninhabitable, humanity has colonized most of the rocky planets and moons of the Solar System. Amid a rising crime rate, the Inter Solar System Police (ISSP) set up a legalized contract system, in which registered bounty hunters (also referred to as "Cowboys") chase criminals and bring them in alive in return for a reward.',
    'outline': 'In 2071, roughly fifty years after an accident with a hyperspace gateway made the Earth almost uninhabitable, humanity has colonized most of the rocky planets and moons of the Solar System. Amid a rising crime rate, the Inter Solar System Police (ISSP) set up a legalized contract system, in which registered bounty hunters (also referred to as "Cowboys") chase criminals and bring them in alive in return for a reward.',
    'lockdata': 'true',
    'dateadded': '2022-10-19 19:46:52',
    'title': 'Cowboy Bebop',
    'originaltitle': 'カウボーイビバップ',
    'writer': 'Hajime Yatate',
    'credits': 'Hajime Yatate',
    'trailer': 'plugin://plugin.video.youtube/?action=play_video&videoid=RI08P5SaJNU',
    'rating': '8.9',
    'year': '1998',
    'mpaa': 'TV-14',
    'imdb_id': 'tt0213338',
    'tmdbid': '30991',
    'premiered': '1998-04-02',
    'releasedate': '1998-04-02',
    'enddate': '1999-04-23',
    'runtime': '25',
    'genre': 'Western',
    'studio': 'WOWOW Prime',
    'tag': 'space western',
    'tvdbid': '76885',
    'art': '',
    'actor': '',
    'id': '76885',
    'episodeguide': '',
    'season': '-1',
    'episode': '-1',
    'status': 'Ended'
}

# Use hset with mapping to add the metadata
r.hset('media:cowboy_bebop', mapping=metadata)

0

In [5]:
# Retrieve metadata
print(r.hgetall('media:cowboy_bebop'))

{'status': 'Ended', 'writer': 'Hajime Yatate', 'year': '1998', 'studio': 'WOWOW Prime', 'mpaa': 'TV-14', 'plot': 'In 2071, roughly fifty years after an accident with a hyperspace gateway made the Earth almost uninhabitable, humanity has colonized most of the rocky planets and moons of the Solar System. Amid a rising crime rate, the Inter Solar System Police (ISSP) set up a legalized contract system, in which registered bounty hunters (also referred to as "Cowboys") chase criminals and bring them in alive in return for a reward.', 'imdb_id': 'tt0213338', 'releasedate': '1998-04-02', 'lockdata': 'true', 'id': '76885', 'credits': 'Hajime Yatate', 'enddate': '1999-04-23', 'episode': '-1', 'premiered': '1998-04-02', 'originaltitle': 'カウボーイビバップ', 'dateadded': '2022-10-19 19:46:52', 'season': '-1', 'runtime': '25', 'rating': '8.9', 'actor': '', 'outline': 'In 2071, roughly fifty years after an accident with a hyperspace gateway made the Earth almost uninhabitable, humanity has colonized most 

Script to process nfo

In [ ]:
# Load and parse the NFO file
tree = ET.parse('tvshow.nfo')
root = tree.getroot()

# Extract metadata from the NFO file
metadata = {}
for child in root:
    metadata[child.tag] = child.text

metadata

Functions

In [36]:
def dfs_index_files_and_folders(root_path):
    nested_index = {}
    analysis_index = {}
    exclude_file_types = json.loads(os.getenv('IGNORED_FILE_TYPES', '[]'))
    excluded_files = json.loads(os.getenv('IGNORED_FILES', '[]'))
    
    def dfs(path, relative_path, layer, index):
        if os.path.isdir(path):
            try:
                contents = os.listdir(path)
                if layer not in index:
                    index[layer] = {}
                index[layer][relative_path] = []
                
                for item in contents:
                    full_path = os.path.join(path, item)
                    relative_item_path = os.path.join(relative_path, item)
                    
                    if os.path.isdir(full_path):
                        index[layer][relative_path].append(item)
                        dfs(full_path, relative_item_path, layer + 1, index)
                    else:
                        if (os.path.splitext(item)[1] not in exclude_file_types) and (item not in excluded_files):
                            index[layer][relative_path].append(item)
            except PermissionError:
                print(f"Permission denied: {path}")

    def contains_season_folder(contents):
        season_pattern = re.compile(r'Season\s*\d+', re.IGNORECASE)
        for item in contents:
            if season_pattern.match(item):
                return True
        return False

    for item in os.listdir(root_path):
        full_path = os.path.join(root_path, item)
        if os.path.isdir(full_path):
            contents = os.listdir(full_path)
            if contains_season_folder(contents):
                dfs(full_path, item, 1, nested_index)
            else:
                dfs(full_path, item, 1, analysis_index)
    
    return nested_index, analysis_index

def add_show_metadata(metadata, metadata_type):
    match metadata_type:
        case 'show':
            r.hset(f'show:{metadata["title"]}', mapping=metadata)
        case 'season':
            r.hset(f'show:{metadata["title"]}:season:{metadata["season"]}', mapping=metadata)
        case 'episode':
            r.hset(f'show:{metadata["title"]}:season:{metadata["season"]}:episode:{metadata["episode"]}', mapping=metadata)
        case _:
            print('Invalid metadata type')
            
def fetch_metadata(nested_index):
    tv = TV()
    season = Season()
    episode = Episode()
    showIndex = nested_index[1]
    episodeIndex = nested_index[2]

    for show in showIndex:
        results = tv.search(show)
        show_id = results[0].id
        show_metadata = tv.details(show_id)
        show_metadata.path = os.path.join(os.getenv("MEDIA_PATH"), show)
        add_show_metadata(show_metadata, 'show')
        
        for season in showIndex[show]:
            seasonNum = season.split(' ')[1]
            season_metadata = season.details(show_id, seasonNum)
            season_metadata.path = os.path.join(show_metadata.path, season)
            add_show_metadata(season_metadata, 'season')
            
            for episode in episodeIndex[os.join(show, season)]:
                episodeNum = episode.split(' ')[1]
                episode_metadata = episode.details(show_id, seasonNum, episodeNum)
                episode_metadata.path = os.path.join(season_metadata.path, episode)
                add_show_metadata(episode_metadata, 'episode')
                    
    if results:
        show_details = tv.details(show_id)
        season_details = season.details(show_id, seasonNum)
        episode_details = episode.details(show_id, seasonNum, episodeNum)

        
        # Prepare metadata dictionary
        metadata = {
            'title': show_details.name,
            'overview': show_details.overview,
            'show_details': show_details,
            'season': seasonNum,
            "season_details": season_details,
            "episode": episodeNum,
            "episode_details": episode_details
            # 'episode_title': episode_details.name,
            # 'episode_overview': episode_details.overview,
            # 'air_date': episode_details.air_date,
            # 'file_path': os.path.join(root, file)
        }
                    
    return metadata

# Directory containing the media files
# media_directory = input("Enter the path to the media directory: ")

In [37]:
api_key = os.getenv('TMDB_API_KEY')
tmdb = TMDb()
tmdb.api_key = api_key
show_title = 'Cowboy Bebop'
seasonNum = 1
episodeNum = 1
tv = TV()

results = tv.search(show_title)
show_id = results[1].id
show_details = tv.details(show_id)

# metadata = fetch_metadata(show_title, seasonNum, episodeNum)
# print(metadata)

In [38]:
show_details.show_path = os.getenv("MEDIA_PATH")

In [40]:
# file_path = os.getenv('MEDIA_PATH')
# result = dfs_index_files_and_folders(file_path)
# with open("burner.txt", "w") as f:
#   print(result, file=f)

In [41]:
# for title in result[1]:
#     clean_title = re.sub(r'\.S\d{2}.*', '', title).replace('.', ' ')
#     print(clean_title)
#     print()